In [1]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

In [2]:
import textnet

In [3]:
#reload(textnet)

In [4]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [5]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_20201124.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,n_sentences,lemmatized_sentences,tlg_epithet,lemmata,lemmata_wordcount
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,3252,"[[ἄρχω, Φοῖβος, παλαιγενεών, κλέος, φώς, μιμνή...",Epici/-ae,"[ἄρχω, Φοῖβος, παλαιγενεών, κλέος, φώς, μιμνήσ...",23231
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,6068,"[[θουκυδιδής, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",Historici/-ae,"[θουκυδιδής, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",72357
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,10245,"[[φιλοσοφία, ἔργον, ἔνειμι, φάσις, βάρβαρος, ἄ...",Biographi,"[φιλοσοφία, ἔργον, ἔνειμι, φάσις, βάρβαρος, ἄρ...",56868
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,1982,"[[ψιθύρισμα, πίτυς, αἰπολέ, τῆνος, παγαισί, με...",Bucolici,"[ψιθύρισμα, πίτυς, αἰπολέ, τῆνος, παγαισί, μελ...",11483
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,152,"[[ῥόδον, δροσοεντά, καταπυκνός, ἐκεινά, ἕρπυλλ...",Bucolici,"[ῥόδον, δροσοεντά, καταπυκνός, ἐκεινά, ἕρπυλλο...",1053


In [43]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [7]:
subcorpora_dict = {
    "Pagan (8th c. BCE - 4th c. CE)" : AGT[(AGT["provenience"]=="pagan") & (AGT["date_avr"] < 4)],
    "Pagan (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")]}

In [8]:
subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "doccount" : doccount, "n_sentences": sents_len,"sentences" : sents})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,doccount,n_sentences,sentences
0,Pagan (8th c. BCE - 4th c. CE),12965219,774,1039409,"[[ἄρχω, Φοῖβος, παλαιγενεών, κλέος, φώς, μιμνή..."
1,Pagan (1st - 4th c. CE),8692448,411,723956,"[[γεωγραφία, σοσσιέ, σενεκιών, ἱστορικοί, διαφ..."
2,Septuagint,799083,55,57856,"[[ἔν, ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ἀό..."
3,Philo of Alexandria,643011,31,82966,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,Titus Flavius Josephus,469580,4,19707,"[[ἱστορία, συγγραφείν, βούλομαι, εἷς, αὐτός, ὁ..."
5,Paul of Tarsus,24066,7,1632,"[[δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπόστολος, ..."
6,New Testament,137787,27,8698,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
7,Christian (1st - 4th c. CE),2975644,131,291084,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."


# term2term & term2doc matraces with sklearn - Paul as example

In [10]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [11]:
len(docs)

1632

In [11]:
AGT_C = subcorpora_dict["Christian (1st - 4th c. CE)"]
len(AGT_C)

131

In [17]:
AGT_C.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,n_sentences,lemmatized_sentences,tlg_epithet,lemmata,lemmata_wordcount
385,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,18288,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...",[],"[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",8667
386,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,11274,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",[],"[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσα...",5306
387,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,1274,"[[ἐπειδηπέρ, πολύς, ἐπιχειρέω, ἀνατάσσομαι, δι...",[],"[ἐπειδηπέρ, πολύς, ἐπιχειρέω, ἀνατάσσομαι, διή...",9263
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,1164,"[[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θε...",[],"[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεό...",7137
389,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,960,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...",[],"[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰη...",9148


In [16]:
author = AGT_C["author"][10]
author

'Paul of Tarsus'

In [19]:
sub_df = AGT_C[AGT_C["author_id"]=="tlg0031john"]
sub_df["date_probs"]

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,n_sentences,lemmatized_sentences,tlg_epithet,lemmata,lemmata_wordcount
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,1164,"[[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θε...",[],"[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεό...",7137
407,tlg0031.tlg023.perseus-grc2.xml,Johnannine literature,New Testament - 1 John,2135,tlg0031john,tlg0031.tlg023,A.D. 1,0.5,{'0.5': 1},0.5,christian,140,"[[ἤν, ἀρχής, ἀκούω, ὁράω, ὀφθαλμός, θεάομαι, χ...",[],"[ἤν, ἀρχής, ἀκούω, ὁράω, ὀφθαλμός, θεάομαι, χε...",941
408,tlg0031.tlg024.perseus-grc2.xml,Johnannine literature,New Testament - 2 John,245,tlg0031john,tlg0031.tlg024,A.D. 1,0.5,{'0.5': 1},0.5,christian,15,"[[πρεσβυτερός, ἐκλεκτῄ, κυρία, τέκνον, ἀγαπάω,...",[],"[πρεσβυτερός, ἐκλεκτῄ, κυρία, τέκνον, ἀγαπάω, ...",113
409,tlg0031.tlg025.perseus-grc2.xml,Johnannine literature,New Testament - 3 John,219,tlg0031john,tlg0031.tlg025,A.D. 1,0.5,{'0.5': 1},0.5,christian,20,"[[πρέσβυς, γαιῴ, ἀγαπητός, ἀγαπάω, ἀληθειᾴ], [...",[],"[πρέσβυς, γαιῴ, ἀγαπητός, ἀγαπάω, ἀληθειᾴ, ἀγα...",111


In [44]:
get_flat_sentences(sub_df["lemmatized_sentences"])

[['βίβλος', 'γένεσις', 'Ἰησοῦς', 'Χριστός', 'υἱός', 'Δαυίδ', 'υἱός', 'Ἀβραάμ'],
 ['Ἀβραάμ',
  'γεννάω',
  'Ἰσαάκ',
  'Ἰσαάκ',
  'γεννάω',
  'ἰακώβ',
  'Ἰακώβ',
  'γεννάω',
  'Ἰούδας',
  'ἀδελφός',
  'Ἰούδας',
  'γεννάω',
  'Φάρες',
  'Ζάρα',
  'Θαμάρ',
  'Φάρες',
  'γεννάω',
  'Ἑσρώμ',
  'Ἑσρώμ',
  'γεννάω',
  'Ἀράμ',
  'Ἀράμ',
  'γεννάω',
  'Ἀμιναδάβ',
  'Ἀμιναδάβ',
  'γεννάω',
  'Ναασσών',
  'Ναασσών',
  'γεννάω',
  'Σαλμών',
  'Σαλμών',
  'γεννάω',
  'βοῦς',
  'Ῥαχάβ',
  'βοῦς',
  'γεννάω',
  'Ἰωβήδ',
  'Ῥούθ',
  'Ἰωβήδ',
  'γεννάω',
  'Ἰεσσαί',
  'Ἰεσσαί',
  'γεννάω',
  'Δαυίδ',
  'βασιλεύς'],
 ['Δαυίδ',
  'γεννάω',
  'Σολομών',
  'Οὐρίας',
  'Σολομών',
  'γεννάω',
  'Ῥοβοάμ',
  'Ῥοβοάμ',
  'γεννάω',
  'Ἀβιά',
  'Ἀβιά',
  'γεννάω',
  'Ἀσάφ',
  'Ἀσάφ',
  'γεννάω',
  'Ἰωσαφάτ',
  'Ἰωσαφάτ',
  'γεννάω',
  'Ἰωράμ',
  'Ἰωράμ',
  'γεννάω',
  'Ὀζίας',
  'Ὀζίας',
  'γεννάω',
  'Ἰωαθάμ',
  'Ἰωαθάμ',
  'γεννάω',
  'ἀχάς',
  'ἀχάς',
  'γεννάω',
  'Ἑζεκίας',
  'Ἑζεκίας',
  'γεννάω',
  'Μανασσῆ

In [35]:
probs_dict = {}
for d in sub_df["date_probs"]:
    probs_dict.update(d)
probs_dict

{'0.5': 1}

In [53]:
authors_dicts = []
for author_id in AGT_C["author_id"].unique():
    sub_df = AGT_C[AGT_C["author_id"]==author_id]
    authors_dict = {}
    authors_dict["author_id"] = author_id
    authors_dict["author"] = sub_df["author"][0]
    authors_dict["n_titles"] = len(sub_df)
    authors_dict["titles"] = sub_df["title"].tolist()
    authors_dict["filenames"] = sub_df["filename"].tolist()
    authors_dict["wordcount"] = sub_df["wordcount"].sum()
    authors_dict["lemmata_wordcount"] = sub_df["lemmata_wordcount"].sum()
    authors_dict["n_sentences"] = sub_df["n_sentences"].sum()
    probs_dict = {}
    for d in sub_df["date_probs"]:
        probs_dict.update(d)
    authors_dict["date_probs"] = probs_dict
    authors_dict["lemmata_lists"] = sub_df["lemmata"].tolist()
    authors_dict["lemmatized_sentences"] = get_flat_sentences(sub_df["lemmatized_sentences"])
    authors_dicts.append(authors_dict)
    
authors_df = pd.DataFrame(authors_dicts)
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences
0,tlg0031a,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8667,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1,tlg0031b,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5306,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18411,2234,{'0.5': 1},"[[ἐπειδηπέρ, πολύς, ἐπιχειρέω, ἀνατάσσομαι, δι...","[[ἐπειδηπέρ, πολύς, ἐπιχειρέω, ἀνατάσσομαι, δι..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8302,1339,{'0.5': 1},"[[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θε...","[[ἔν, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θε..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11119,1632,{'0.5': 1},"[[δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπόστολος, ...","[[δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπόστολος, ..."
